In [1]:
%load_ext autoreload
%autoreload 2
from expressiveness_benchmark.types import Plan, Task, Language, SourceRange, Program
from code_widget.example import CodeWidget
from dataclasses import replace
import json
import pandas as pd

In [2]:
# CHANGE ME!
TASK_ID = 'strings_to_numbers'
AUTHOR = 'will'

In [17]:
task = Task(
    id=TASK_ID,
    category="Strings",
    name="Convert strings with different formats to numbers",
    description="Convert the string value in each row into a number, removing \
commas if format is \"comma_sep\" and underscores if format is \"under_sep\".",
    plan=[
        Plan(id="iter", description="Iterate over rows"),
        Plan(id="cond", description="Check format"),
        Plan(id="clean", description="Remove separator"),
        Plan(id="number", description="Convert to number")
    ],
    sample_input={
        "numbers": [
            {"format": "comma_sep", "value": "12,337,800"},
            {"format": "under_sep", "value": "80_999"}
        ]
    },
    sample_output=[12337800, 80999],
)
task.save()

prototype = Program(
    task=TASK_ID,
    author=AUTHOR,
    language=''    
)

In [18]:
sql = replace(prototype,
    language='sql',
    source='''SELECT 
  CAST(
    REPLACE(
      value, 
      CASE format 
        WHEN "comma_sep" THEN "," 
        WHEN "under_sep" THEN "_" 
      END, "")
    AS integer)
FROM numbers''').load_plan()
sql.execute(task)
sql.save()

In [19]:
sql.widget(task)

Output()

CodeWidget(program='{"task": "strings_to_numbers", "language": "sql", "plan": {"cond": [{"line": 1, "start": 8…

In [20]:
datalog = replace(prototype,
    language='datalog',
    source='''.decl clean(Format:symbol, Inp:symbol, I:number, Outp:symbol) 
clean(Format, Inp, 0, "") :- numbers(Format, Inp).
clean(Format, Inp, I+1, Outp) :-
  clean(Format, Inp, I, Outp_rec),
  I <= strlen(Inp),
  Chr = substr(Inp, I, 1),
  ((Format = "comma_sep", Sep = ",");
   (Format = "under_sep", Sep = "_")),
  ((Chr  = Sep, Outp = Outp_rec);
   (Chr != Sep, Outp = cat(Outp_rec, Chr))).

strings_to_numbers(N) :-
  clean(Format, Inp, strlen(Inp), Outp),
  N = to_number(Outp).''').load_plan()
datalog.execute(task)
datalog.save()

In [21]:
datalog.widget(task)

Output()

CodeWidget(program='{"task": "strings_to_numbers", "language": "datalog", "plan": {"number": [{"line": 13, "st…

In [22]:
pandas = replace(prototype,
    language='python-pandas',
    source='''def strings_to_numbers(numbers):
  def convert(row):
    sep = "," if row.format == 'comma_sep' else "_"
    return int(row.value.replace(sep, ""))
  return numbers.apply(convert, axis=1).tolist()''').load_plan()
pandas.execute(task)
pandas.save()

In [23]:
pandas.widget(task)

Output()

CodeWidget(program='{"task": "strings_to_numbers", "language": "python-pandas", "plan": {"cond": [{"line": 2, …

In [28]:
imperative = replace(prototype,
    language='python-imperative',
    source='''def strings_to_numbers(numbers):
  output = []
  for row in numbers:
    if row["format"] == 'comma_sep':
      sep = ","
    else:
      sep = "_"
    output.append(int(row["value"].replace(sep, "")))
  return output''').load_plan()
imperative.execute(task)
imperative.save()

In [29]:
imperative.widget(task)

Output()

CodeWidget(program='{"task": "strings_to_numbers", "language": "python-imperative", "plan": {}, "source": "def…

In [32]:
functional = replace(prototype,
    language='python-functional',
    source='''def strings_to_numbers(numbers):
  return [
    int(row["value"].replace(
      "," if row["format"] == "comma_sep" else "_", ""))
    for row in numbers
  ]''').load_plan()
functional.execute(task)
functional.save()

In [33]:
functional.widget(task)

Output()

CodeWidget(program='{"task": "strings_to_numbers", "language": "python-functional", "plan": {"iter": [{"line":…